In [ ]:
# @title load data and split to train, validataion, and test data set

import tensorflow as tf
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from tensorflow import keras
import random

key = "img_224"
data_dir = "_"
model_dir = " "
figure_dir = " "

train = np.load(data_dir+"train.npz")
test = np.load(data_dir+"test.npz")

X = train[key]
y = train["labels"].reshape(-1)

test_X = test[key]
test_y = test["labels"].reshape(-1)


indices = np.arange(len(X))
np.random.seed(123)
np.random.shuffle(indices)
X = X[indices]
y = y[indices]
age = np.minimum(train["ages"].reshape(-1)[indices] // 20, 4)

combined_data = np.column_stack((y, age))
stratified_splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2,  random_state=123)

# Perform stratified split
for train_index, val_index in stratified_splitter.split(combined_data, combined_data[:, 0]):
    train_img, valid_img = X[train_index], X[val_index]
    train_labels, valid_labels = y[train_index], y[val_index]

t_total  = len(train_labels)
t_P = train_labels.sum()
t_N = t_total - t_P

v_total = len(valid_labels)
v_P = valid_labels.sum()
v_N = v_total - v_P

print("X_train shape:", train_img.shape)
print("y_train shape:", train_labels.shape)
print("X_val shape:", valid_img.shape)
print("y_val shape:", valid_labels.shape)
print("train P:{:.4f}, N:{:.4f}, total:{}".format(t_P/t_total, t_N/t_total, t_total))
print("valid P:{:.4f}, N:{:.4f}, total:{}".format(v_P/v_total, v_N/v_total, v_total))

X_train shape: (2317, 224, 224, 3)
y_train shape: (2317,)
X_val shape: (580, 224, 224, 3)
y_val shape: (580,)
train P:0.5585, N:0.4415, total:2317
valid P:0.5586, N:0.4414, total:580


In [ ]:
# @title evaluate
from sklearn.metrics import *
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sklearn


class evaluate:

  def __init__(self, labels, predict, val_labels, val_predict, sex, age):
    self.labels = labels.squeeze()
    self.predict = predict.squeeze()
    self.val_labels = val_labels.squeeze()
    self.val_predict = val_predict.squeeze()
    self.figsize = (10, 10)
    self.threshold = 0.5
    self.sex = sex
    self.age = age

  def set_threshold(self):
    # calculate Youden’s index in valid set
    val_label = self.val_labels.squeeze()
    val_predict = self.val_predict.squeeze()
    def calculate_Y_index(threshold):
      TP = np.logical_and(val_label > threshold, val_predict >= threshold).sum()
      TN = np.logical_and(val_label < threshold, val_predict <  threshold).sum()
      FP = np.logical_and(val_label < threshold, val_predict >= threshold).sum()
      FN = np.logical_and(val_label > threshold, val_predict <  threshold).sum()

      assert TP + FN + TN + FP == len(val_label)
      assert TP + FN == val_label.sum()
      assert TN + FP == len(val_label) - val_label.sum()
      Sensitivity = TP / (TP + FN)
      Specificity = TN / (FP + TN)
      return Sensitivity + Specificity

    thresholds = np.sort(val_predict)[1:-1]
    y = np.vectorize(calculate_Y_index)(thresholds)
    self.threshold = thresholds[np.argmax(y)]


  def result(self):
    threshold = self.threshold
    acc = accuracy_score(self.labels, self.predict>=threshold)
    auc = roc_auc_score(self.labels, self.predict)

    TP = np.logical_and(self.labels > threshold, self.predict >= threshold).sum()
    TN = np.logical_and(self.labels < threshold, self.predict < threshold).sum()
    FP = np.logical_and(self.labels < threshold, self.predict >= threshold).sum()
    FN = np.logical_and(self.labels > threshold, self.predict < threshold).sum()

    assert TP + FN == self.labels.sum()
    assert TN + FP == self.labels.shape[0] - self.labels.sum()

    Sensitivity = round(TP / (TP + FN), 4)
    Specificity = round(TN / (FP + TN), 4)
    if TP + FP == 0 or TN + FN == 0:
      PPV = None
      NPV = None
    else:
      PPV = round(TP / (TP + FP), 4)
      NPV = round(TN / (TN + FN), 4)

    return (Sensitivity, Specificity, PPV, NPV, acc, auc)

  def plot_save_roc(self, name, filename, **kwargs):
    fp, tp, _ = sklearn.metrics.roc_curve(self.labels, self.predict)
    plt.figure(figsize=self.figsize)
    plt.plot(100*fp, 100*tp, label=name, linewidth=4, **kwargs)
    plt.xlabel('False positives [%]', fontsize=20)
    plt.ylabel('True positives [%]', fontsize=20)
    plt.grid(True)
    plt.xlim(0, 100)
    plt.ylim(0, 100)
    plt.plot([0,100], [0,100], linestyle = '--', color = 'k')
    ax = plt.gca()
    ax.set_aspect(aspect=1)
    plt.legend()
    plt.savefig(filename)

  def plot_cm(self, filename, **kwargs):

    cm = confusion_matrix(self.labels, self.predict > self.threshold)
    plt.figure(figsize=(5,5))
    sns.heatmap(cm, annot=True, fmt="d",**kwargs)

    plt.ylabel('Actual Values', fontsize=20)
    plt.xlabel('Predicted Values', fontsize=20)

    print('Legitimate Transactions Detected (True Negatives): ', cm[0][0])
    print('Legitimate Transactions Incorrectly Detected (False Positives): ', cm[0][1])
    print('Fraudulent Transactions Missed (False Negatives): ', cm[1][0])
    print('Fraudulent Transactions Detected (True Positives): ', cm[1][1])
    print('Total Fraudulent Transactions: ', np.sum(cm[1]))
    plt.savefig(filename)

In [ ]:
neg = t_N
pos = t_P
total = neg+pos

weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 1.13
Weight for class 1: 0.90


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential, regularizers
from tensorflow.keras.initializers import Constant, glorot_normal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2, L1L2
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os
from datetime import datetime

METRICS = [
      keras.metrics.AUC(name='auc',num_thresholds=1000),
]

now = datetime.now()
dt_string = now.strftime("%d_%m_%H_%M")
print(dt_string)
checkpoint_path = dt_string+"/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
callbacks=[ModelCheckpoint(filepath=checkpoint_path,
                           save_weights_only=True,
                           verbose=0,
                           save_freq="epoch"),
           EarlyStopping(monitor='val_auc',
                         verbose=1,
                         patience=4,
                         mode='max',
                         min_delta = 0.002,
                         restore_best_weights=True),
          ]

initial_bias = np.log([pos/neg])

def get_vgg16(number_Dence, dropout_rate, seed=0,input_shape=(224, 224, 3), output_bias= None):
    if output_bias is not None:
        output_bias = Constant(output_bias)
    inputs = keras.Input(shape=input_shape)
    x = keras.applications.vgg16.preprocess_input(inputs)
    x = keras.applications.VGG16(input_tensor=x,
                               include_top=False,
                               weights="imagenet")
    x.trainable = False

    x = layers.Flatten()(x.output)
    x = layers.Dropout(dropout_rate)(x)
    if number_Dence > 1:
        x = layers.Dense(number_Dence, activation='relu', kernel_regularizer=L1L2(l1=0.005, l2=0.005), kernel_initializer=glorot_normal(seed=seed))(x)
    outputs = keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=l2(l2=0.001), kernel_initializer=glorot_normal(seed=seed), bias_initializer=output_bias)(x)
    return  keras.Model(inputs, outputs)


11_04_15_05


In [ ]:


# datetime object containing current date and time

lr = 2e-3
dr = 0.3
seed = 123
model = get_vgg16(0, 0.2, seed=seed,input_shape=(224, 224, 3), output_bias= initial_bias)




train_ds = tf.data.Dataset.from_tensor_slices((train_img, train_labels))
val_ds = tf.data.Dataset.from_tensor_slices((valid_img, valid_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_X, test_y))

BATCH_SIZE = 32

train_ds = train_ds.batch(BATCH_SIZE)
val_ds = val_ds.batch(32)
test_ds = test_ds.batch(32)

data_augmentation = keras.Sequential([
    layers.RandomZoom(0.05, fill_mode="constant", fill_value=255, seed = seed+1),
    layers.RandomTranslation((0, 0), width_factor=(-0.2, 0.2), fill_mode="constant", fill_value=255, seed=seed+2),
    #layers.RandomCrop(224, 224),
])
train_ds_augmented = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y)).prefetch(
        tf.data.AUTOTUNE)



optimizer = Adam(learning_rate=lr)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=METRICS)

results = model.evaluate(test_ds, verbose=1)
history = model.fit(train_ds_augmented, epochs=10,
                    validation_data=val_ds,
                    callbacks=callbacks,
                    class_weight=class_weight)


In [ ]:
test_predict = model.predict(test_X)
val_predict = model.predict(valid_img)

# model3 = evaluate(train_labels, train_predict, valid_labels, val_predict, test_sex, test_age)
model3 = evaluate(test_y, test_predict, valid_labels, val_predict, test["sexs"].squeeze(), test["ages"].squeeze())
Sensitivity, Specificity, PPV, NPV, acc, auc = model3.result()
print(Sensitivity, Specificity, PPV, NPV, acc, auc)
model3.set_threshold()
Sensitivity, Specificity, PPV, NPV, acc, auc = model3.result()
print(Sensitivity, Specificity, PPV, NPV, acc, auc)
model3.threshold

In [ ]:
model.save(model_dir+"new_freeze")


In [ ]:
#model = keras.models.load_model(model_dir+"new_freeze")
print("Number of layers in the base model: ", len(model.layers))
model.trainable = True


Number of layers in the base model:  25


In [ ]:
for layer in model.layers[:-8]:
    layer.trainable = False
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.__operators__.getitem_2  (None, 224, 224, 3)       0         
  (SlicingOpLambda)                                              
                                                                 
 tf.nn.bias_add_2 (TFOpLamb  (None, 224, 224, 3)       0         
 da)                                                             
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0   

In [ ]:
model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=1e-6),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=METRICS)
initial_epoch = 10
total_epoch = initial_epoch + 20
history_fined = model.fit(train_ds_augmented, epochs=total_epoch,
                          initial_epoch=initial_epoch,
                          validation_data=val_ds, callbacks=callbacks, class_weight=class_weight)

In [ ]:
model.load_weights(checkpoint_path.format(epoch=16))

In [ ]:
test_predict = model.predict(test_X)
val_predict = model.predict(valid_img)

# model3 = evaluate(train_labels, train_predict, valid_labels, val_predict, test_sex, test_age)
model3 = evaluate(test_y, test_predict, valid_labels, val_predict, test["sexs"].squeeze(), test["ages"].squeeze())
Sensitivity, Specificity, PPV, NPV, acc, auc = model3.result()
print(Sensitivity, Specificity, PPV, NPV, acc, auc)
model3.set_threshold()
Sensitivity, Specificity, PPV, NPV, acc, auc = model3.result()
print(Sensitivity, Specificity, PPV, NPV, acc, auc)
model3.threshold

19/19 [==============================] - 2s 123ms/step
0.7792 0.8489 0.8825 0.7254 0.8076062639821029 0.880432303545511
0.7057 0.9038 0.9144 0.6784 0.7863534675615212 0.880432303545511


0.712806

In [ ]:
model.save(model_dir+"new_finetuning")